In [1]:
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
import pandas as pd
import numpy as np
from pandasql import sqldf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import mysql.connector as mysql_drive
from dotenv import load_dotenv
import os
load_dotenv()

# Set Pandas options to display more columns
pd.options.display.max_columns=150
import warnings
warnings.filterwarnings("ignore")

#Weighted Mean Absolute Percentage Error
def wape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()
def wbpe(y_true, y_pred):
    return (y_true - y_pred).sum() / np.abs(y_true).sum()
def wcpe(wape, wbpe, beta):
    a = 1 - np.clip(wape, a_min = 0, a_max = 1)
    b = 1 - min(1, np.abs(wbpe))
    return ((1 + beta**2) * a * b) / (beta**2 * a + b)

In [2]:
#days_ago_60 = str((datetime.now() - timedelta(60) - timedelta(35)).date()) # - timedelta(21) bisa diubah agar jadi '2023-05-31'
#days_ago_37 = str((datetime.now() - timedelta(44) - timedelta(35)).date())
#days_ago_30 = str((datetime.now() - timedelta(30) - timedelta(35)).date())
#days_ago_14 = str((datetime.now() - timedelta(14) - timedelta(35)).date())
#days_ago_1 = str((datetime.now() - timedelta(1)   - timedelta(35)).date())

In [2]:
try:
    mydb = mysql_drive.connect(host=os.getenv("HOST"), user=os.getenv("DATABASE_USER"),port=os.getenv("DATABASE_PORT"),
                                database=os.getenv("DATABASE_NAME"), password=os.getenv("DATABASE_PASSWORD"))
    mycursor = mydb.cursor()
    mycursor.execute('''select max(date_format(sk_created_date,'%Y-%m-%d')) as last_date
    from digital_leads.digital_leads''')
    last_date = mycursor.fetchone()
except Exception as e:
    mydb.close()
    print(str(e))
print("Last Database date  : " , last_date)

Last Database date  :  ('2023-07-27',)


In [3]:
days_ago_1_str  = ''.join(last_date)
days_ago_1 = datetime.strptime(days_ago_1_str, '%Y-%m-%d').date()
days_ago_14 = days_ago_1 - timedelta(days = 13)
days_ago_30 = days_ago_1 - timedelta(days = 29)
days_ago_44 = days_ago_1 - timedelta(days = 43)
days_ago_60 = days_ago_1 - timedelta(days = 59)

print("days_ago_14 : " , days_ago_14)
print("days_ago_30 : " , days_ago_30)
print("days_ago_44 : " , days_ago_44)
print("days_ago_60 : " , days_ago_60)

days_ago_14 :  2023-07-14
days_ago_30 :  2023-06-28
days_ago_44 :  2023-06-14
days_ago_60 :  2023-05-29


In [4]:
try:
    mydb = mysql_drive.connect(host=os.getenv("HOST"), user=os.getenv("DATABASE_USER"),port=os.getenv("DATABASE_PORT"),
                                database=os.getenv("DATABASE_NAME"), password=os.getenv("DATABASE_PASSWORD"))
    query = '''
with google As (
Select date_format(created_date,'%Y-%m-%d') as created_date,
	CASE WHEN lower(googleAdsCampaignName) LIKE '%mobil%' THEN 'BPKB Mobil'
		 WHEN lower(googleAdsCampaignName) LIKE '%motor%' THEN 'BPKB Motor'
		 WHEN lower(googleAdsCampaignName) LIKE '%rumah%' THEN 'Sertifikat Rumah'
		 WHEN lower(googleAdsCampaignName) LIKE '%all product%' THEN 'All Product'
		 WHEN lower(googleAdsCampaignName) LIKE '%all-product%' THEN 'All Product'
		 else 'Others'
		 END AS product,
		googleAdsCampaignName as campaign, 
		sum(advertiserAdClicks) as clicks, sum(advertiserAdCost) as cost
	from digital_leads.google_ads_metrics
	where googleAdsCampaignName <> '(not set)'
			and date_format(created_date,'%Y-%m-%d') <= '{0}'
	group by created_date, googleAdsCampaignName, product
),
leads As (
select utm_original, product, date_format(sk_created_date,'%Y-%m-%d') as leads_date, sum(funnel_leads) as leads
    from digital_leads.digital_leads
    where submissionid is not null and group_utm_source = 'Google'
          and product in ('BPKB Mobil', 'BPKB Motor', 'Sertifikat Rumah')
          and sk_created_date > 1 and utm_original is not null
    group by utm_original, product, sk_created_date
),
prospect As (
select utm_original, product, date_format(sk_closing_date,'%Y-%m-%d') as prospect_date, sum(funnel_prospect) as prospect
       from digital_leads.digital_leads
       where submissionid is not null and group_utm_source = 'Google'
             and product in ('BPKB Mobil', 'BPKB Motor', 'Sertifikat Rumah')
             and sk_closing_date > 1 and utm_original is not null
       group by utm_original, product, sk_closing_date                
),
funding As (
select utm_original, product, date_format(sk_golive_date,'%Y-%m-%d') as funding_date, sum(funnel_funding) as funding,
		sum(case when funnel_funding = 1 then fundingamount  else 0 end) as NTF
       from digital_leads.digital_leads
       where submissionid is not null and group_utm_source = 'Google'
             and product in ('BPKB Mobil', 'BPKB Motor', 'Sertifikat Rumah')
             and sk_golive_date > 1 and utm_original is not null
       group by utm_original, product, sk_golive_date
),
g_l As (
select g.created_date, g.product, g.campaign, g.clicks, g.cost, l.leads
	from google as g
	left outer join leads as l on g.created_date = l.leads_date
						and g.campaign = l.utm_original
						and g.product = l.product
	where g.product not in ('All Product', 'Others')
union 
select l.leads_date as created_date, l.product, l.utm_original as campaign, g.clicks, g.cost, l.leads
	from google as g
	right outer join leads as l on g.created_date = l.leads_date
						and g.campaign = l.utm_original
						and g.product = l.product
	where g.product not in ('All Product', 'Others')
)
,
g_lp As (
select gl.created_date, gl.product, gl.campaign, gl.clicks, gl.cost, gl.leads, p.prospect
	from g_l as gl
	left outer join prospect as p on gl.created_date = p.prospect_date
						and gl.campaign = p.utm_original
						and gl.product = p.product
union 
select p.prospect_date as created_date, p.product, p.utm_original as campaign, gl.clicks, gl.cost, gl.leads, p.prospect
	from g_l as gl
	right outer join prospect as p on gl.created_date = p.prospect_date
						and gl.campaign = p.utm_original
						and gl.product = p.product
)
select glp.created_date, glp.product, glp.campaign, glp.clicks, glp.cost, glp.leads, glp.prospect, f.funding, f.NTF
	from g_lp as glp
	left outer join funding as f on glp.created_date = f.funding_date
						and glp.campaign = f.utm_original
						and glp.product = f.product
union 
select f.funding_date as created_date, f.product, f.utm_original as campaign, glp.clicks, glp.cost, glp.leads, glp.prospect,
		f.funding, f.NTF
	from g_lp as glp
	right outer join funding as f on glp.created_date = f.funding_date
						and glp.campaign = f.utm_original
						and glp.product = f.product;     
    '''.format(days_ago_1)
    google = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))
#google.head()

In [5]:
try:
    mydb = mysql_drive.connect(host=os.getenv("HOST"), user=os.getenv("DATABASE_USER"),port=os.getenv("DATABASE_PORT"),
                                database=os.getenv("DATABASE_NAME"), password=os.getenv("DATABASE_PASSWORD"))
    query = '''
        select googleAdsCampaignName as campaign,
            CASE WHEN lower(googleAdsCampaignName) LIKE '%mobil%' THEN 'BPKB Mobil'
		        WHEN lower(googleAdsCampaignName) LIKE '%motor%' THEN 'BPKB Motor'
		        WHEN lower(googleAdsCampaignName) LIKE '%rumah%' THEN 'Sertifikat Rumah'
		        WHEN lower(googleAdsCampaignName) LIKE '%all product%' THEN 'All Product'
		        WHEN lower(googleAdsCampaignName) LIKE '%all-product%' THEN 'All Product'
		        else 'Others'
		        END AS product,
            count(case when (created_date <= '{1}')then 1 end) as count_train,
            count(case when (created_date >= '{1}')then 1 end) as count_valid,
            min(date_format(created_date,'%Y-%m-%d')) as min, max(date_format(created_date,'%Y-%m-%d')) as max
        from digital_leads.google_ads_metrics
        where date_format(created_date,'%Y-%m-%d') <= '{3}' and googleAdsCampaignName <> '(not set)'
        group by googleAdsCampaignName, product
        having  min(date_format(created_date,'%Y-%m-%d')) <= '{0}' 
                and max(date_format(created_date,'%Y-%m-%d')) > '{2}'
                and count(case when (created_date <= '{1}')then 1 end) > 2
                and count(case when (created_date >= '{1}')then 1 end) > 2
    '''.format(days_ago_44,days_ago_30, days_ago_14,days_ago_1)
    active = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))
active

,campaign,product,count_train,count_valid,min,max
0,Leads | BPKB Mobil | Discovery | Web 1 | Natio...,BPKB Mobil,63,26,2023-02-01,2023-07-27
1,Leads | BPKB Mobil | Search | Web 1 | Jabarteng,BPKB Mobil,10,29,2023-06-06,2023-07-27
2,Leads | BPKB Mobil | Search | Web 1 | Jabodeta...,BPKB Mobil,18,30,2023-05-17,2023-07-27
3,Leads | BPKB Mobil | Search | Web 1 | Nationwi...,BPKB Mobil,119,30,2023-02-01,2023-07-27
4,Leads | BPKB Motor | Discovery | Web 1 | Natio...,BPKB Motor,58,8,2023-02-01,2023-07-27
5,Leads | BPKB Motor | Search | Web 1 | Jabarteng,BPKB Motor,22,28,2023-06-05,2023-07-27
6,Leads | BPKB Motor | Search | Web 1 | Jabodetabek,BPKB Motor,24,29,2023-06-05,2023-07-27
7,Leads | BPKB Motor | Search | Web 1 | Nationwi...,BPKB Motor,20,4,2023-05-01,2023-07-20
8,Leads | Rumah | Search | Web 1 | Jabodetabek,Sertifikat Rumah,24,30,2023-06-05,2023-07-27
9,Leads | Seritifikat Rumah | Discovery | Web 1 ...,Sertifikat Rumah,60,15,2023-02-01,2023-07-27


In [6]:
Active_campaign = google.loc[google.campaign.isin(active.campaign.values)]
               #, ['googleAdsCampaignName', 'created_date', 'advertiserAdClicks', 'cost']]
Active_campaign['created_date'] = pd.to_datetime(Active_campaign['created_date']).dt.date
#Active_campaign.campaign.unique()
#Active_campaign.head()
#Active_campaign.info()

In [67]:
train_click = Active_campaign.loc[Active_campaign['created_date'] < days_ago_30, ['campaign', 'created_date', 'clicks']]
valid_click = Active_campaign.loc[(Active_campaign['created_date'] >= days_ago_30) & (Active_campaign['created_date'] < days_ago_1), ['campaign', 'created_date', 'clicks']]
train_cost = Active_campaign.loc[Active_campaign['created_date'] < days_ago_30, ['campaign', 'created_date', 'cost']]
valid_cost = Active_campaign.loc[(Active_campaign['created_date'] >= days_ago_30) & (Active_campaign['created_date'] < days_ago_1), ['campaign', 'created_date', 'cost']]
train_leads = Active_campaign.loc[Active_campaign['created_date'] < days_ago_30, ['campaign', 'created_date', 'leads']]
valid_leads = Active_campaign.loc[(Active_campaign['created_date'] >= days_ago_30) & (Active_campaign['created_date'] < days_ago_1), ['campaign', 'created_date', 'leads']]
train_prospect = Active_campaign.loc[Active_campaign['created_date'] < days_ago_30, ['campaign', 'created_date', 'prospect']]
valid_prospect = Active_campaign.loc[(Active_campaign['created_date'] >= days_ago_30) & (Active_campaign['created_date'] < days_ago_1), ['campaign', 'created_date', 'prospect']]
train_funding = Active_campaign.loc[Active_campaign['created_date'] < days_ago_30, ['campaign', 'created_date', 'funding']]
valid_funding = Active_campaign.loc[(Active_campaign['created_date'] >= days_ago_30) & (Active_campaign['created_date'] < days_ago_1), ['campaign', 'created_date', 'funding']]
train_NTF = Active_campaign.loc[Active_campaign['created_date'] < days_ago_30, ['campaign', 'created_date', 'NTF']]
valid_NTF = Active_campaign.loc[(Active_campaign['created_date'] >= days_ago_30) & (Active_campaign['created_date'] < days_ago_1), ['campaign', 'created_date', 'NTF']]

train_click = train_click.rename(columns={'created_date': 'ds', 'clicks': 'y', 'campaign': 'campaign'})
valid_click = valid_click.rename(columns={'created_date': 'ds', 'clicks': 'y', 'campaign': 'campaign'})
train_cost = train_cost.rename(columns={'created_date': 'ds', 'cost': 'y', 'campaign': 'campaign'})
valid_cost = valid_cost.rename(columns={'created_date': 'ds', 'cost': 'y', 'campaign': 'campaign'})
train_leads = train_leads.rename(columns={'created_date': 'ds', 'leads': 'y', 'campaign': 'campaign'})
valid_leads = valid_click.rename(columns={'created_date': 'ds', 'leads': 'y', 'campaign': 'campaign'})
train_prospect = train_prospect.rename(columns={'created_date': 'ds', 'prospect': 'y', 'campaign': 'campaign'})
valid_prospect = valid_prospect.rename(columns={'created_date': 'ds', 'prospect': 'y', 'campaign': 'campaign'})
train_funding = train_funding.rename(columns={'created_date': 'ds', 'funding': 'y', 'campaign': 'campaign'})
valid_funding = valid_funding.rename(columns={'created_date': 'ds', 'funding': 'y', 'campaign': 'campaign'})
train_NTF = train_NTF.rename(columns={'created_date': 'ds', 'NTF': 'y', 'campaign': 'campaign'})
valid_NTF = valid_NTF.rename(columns={'created_date': 'ds', 'NTF': 'y', 'campaign': 'campaign'})

train_click['ds']= pd.to_datetime(train_click['ds'])
valid_click['ds']= pd.to_datetime(valid_click['ds'])
train_cost['ds']= pd.to_datetime(train_cost['ds'])
valid_cost['ds']= pd.to_datetime(valid_cost['ds'])
train_leads['ds']= pd.to_datetime(train_leads['ds'])
valid_leads['ds']= pd.to_datetime(valid_leads['ds'])
train_prospect['ds']= pd.to_datetime(train_prospect['ds'])
valid_prospect['ds']= pd.to_datetime(valid_prospect['ds'])
train_funding['ds']= pd.to_datetime(train_funding['ds'])
valid_funding['ds']= pd.to_datetime(valid_funding['ds'])
train_NTF['ds']= pd.to_datetime(train_NTF['ds'])
valid_NTF['ds']= pd.to_datetime(valid_NTF['ds'])

In [42]:
hist_click = train_click.loc[train_click['ds'].dt.date >= days_ago_60]
hist_cost = train_cost.loc[train_cost['ds'].dt.date >= days_ago_60]
hist_leads = train_leads.loc[train_leads['ds'].dt.date >= days_ago_60]
hist_prospect = train_prospect.loc[train_prospect['ds'].dt.date >= days_ago_60]
hist_funding = train_funding.loc[train_funding['ds'].dt.date >= days_ago_60]
hist_NTF = train_NTF.loc[train_NTF['ds'].dt.date >= days_ago_60]

In [43]:
q = """with exclusion as (SELECT campaign, sum(y) FROM train_click group by campaign having sum(y) is null
       ) select * from train_click where campaign not in (SELECT campaign from exclusion);"""
train_click = sqldf(q)

In [44]:
q = """with exclusion as (SELECT campaign, sum(y) FROM train_cost group by campaign having sum(y) is null
       ) select * from train_cost where campaign not in (SELECT campaign from exclusion);"""
train_cost = sqldf(q)

In [45]:
q = """with exclusion as (SELECT campaign, sum(y) FROM train_leads group by campaign having sum(y) is null
       ) select * from train_leads where campaign not in (SELECT campaign from exclusion);"""
train_leads = sqldf(q)

In [46]:
q = """with exclusion as (SELECT campaign, sum(y) FROM train_prospect group by campaign having sum(y) is null
       ) select * from train_prospect where campaign not in (SELECT campaign from exclusion);"""
train_prospect = sqldf(q)

In [47]:
q = """with exclusion as (SELECT campaign, sum(y) FROM train_funding group by campaign having sum(y) is null
       ) select * from train_funding where campaign not in (SELECT campaign from exclusion);"""
train_funding = sqldf(q)

In [48]:
q = """with exclusion as (SELECT campaign, sum(y) FROM train_NTF group by campaign having sum(y) is null
       ) select * from train_NTF where campaign not in (SELECT campaign from exclusion);"""
train_NTF = sqldf(q)

In [49]:
click = list()
for campaign in train_click['campaign'].unique():
    print('campaign:', campaign)
    train_ = train_click.loc[train_click['campaign'] == campaign]
    valid_ = valid_click.loc[valid_click['campaign'] == campaign]

    m = Prophet(seasonality_mode='additive', yearly_seasonality=False, 
                weekly_seasonality=True, daily_seasonality=True
               )
    if train_.shape[0] > 2:
        m.fit(train_)
    
    future = m.make_future_dataframe(periods=60, include_history=False)
    future = future.merge(valid_[['ds']], on='ds', how='left')
    forecast = m.predict(future)
    forecast['campaign'] = campaign
    click.append(forecast[['ds', 'yhat', 'campaign']])
    
click = pd.concat(click, ignore_index=True)
click['yhat'] = click['yhat'].clip(lower=0)
click = click.merge(valid_click, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(click['y'], click['yhat']))
print("WBPE : " , wbpe(click['y'], click['yhat']))
print("WCPE : " , wcpe(wape(click['y'], click['yhat']), wbpe(click['y'], click['yhat']),2))

15:25:03 - cmdstanpy - INFO - Chain [1] start processing
15:25:03 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time


15:25:03 - cmdstanpy - INFO - Chain [1] start processing
15:25:03 - cmdstanpy - INFO - Chain [1] done processing
15:25:03 - cmdstanpy - INFO - Chain [1] start processing
15:25:03 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time


15:25:04 - cmdstanpy - INFO - Chain [1] start processing


...

campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek


15:25:04 - cmdstanpy - INFO - Chain [1] start processing
15:25:04 - cmdstanpy - INFO - Chain [1] done processing
15:25:05 - cmdstanpy - INFO - Chain [1] start processing
15:25:05 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng


15:25:05 - cmdstanpy - INFO - Chain [1] start processing
15:25:05 - cmdstanpy - INFO - Chain [1] done processing


WAPE :  0.5827759269376561
WBPE :  -0.2692994593624651
WCPE :  0.6352439117592736


In [50]:
cost = list()
for campaign in train_cost['campaign'].unique():
    print('campaign:', campaign)
    train_ = train_cost.loc[train_cost['campaign'] == campaign]
    valid_ = valid_cost.loc[valid_cost['campaign'] == campaign]
    
    m = Prophet(seasonality_mode='additive', yearly_seasonality=False, 
                weekly_seasonality=True, daily_seasonality=True
               )
    m.fit(train_)
    
    future = m.make_future_dataframe(periods=60, include_history=False)
    future = future.merge(valid_[['ds']], on='ds', how='left')
    forecast = m.predict(future)
    forecast['campaign'] = campaign
    cost.append(forecast[['ds', 'yhat', 'campaign']])
    
cost = pd.concat(cost, ignore_index=True)
cost['yhat'] = cost['yhat'].clip(lower=0)
cost = cost.merge(valid_cost, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(cost['y'], cost['yhat']))
print("WBPE : " , wbpe(cost['y'], cost['yhat']))
print("WCPE : " , wcpe(wape(cost['y'], cost['yhat']), wbpe(cost['y'], cost['yhat']),2))

15:25:06 - cmdstanpy - INFO - Chain [1] start processing
15:25:06 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time


15:25:06 - cmdstanpy - INFO - Chain [1] start processing
15:25:07 - cmdstanpy - INFO - Chain [1] done processing
15:25:07 - cmdstanpy - INFO - Chain [1] start processing
15:25:07 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time


15:25:07 - cmdstanpy - INFO - Chain [1] start processing
15:25:07 - cmdstanpy - INFO - Chain [1] done processing


...

campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek


15:25:08 - cmdstanpy - INFO - Chain [1] start processing
15:25:08 - cmdstanpy - INFO - Chain [1] done processing
15:25:08 - cmdstanpy - INFO - Chain [1] start processing
15:25:08 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng


15:25:08 - cmdstanpy - INFO - Chain [1] start processing
15:25:08 - cmdstanpy - INFO - Chain [1] done processing


WAPE :  0.6827424661968873
WBPE :  -0.23023600582880752
WCPE :  0.5989163468775912


In [51]:
leads = list()
for campaign in train_leads['campaign'].unique():
    print('campaign:', campaign)
    train_ = train_leads.loc[train_leads['campaign'] == campaign]
    valid_ = valid_leads.loc[valid_leads['campaign'] == campaign]
    
    m = Prophet(seasonality_mode='additive', yearly_seasonality=False, 
                weekly_seasonality=True, daily_seasonality=True
               )
    m.fit(train_)
    
    future = m.make_future_dataframe(periods=60, include_history=False)
    future = future.merge(valid_[['ds']], on='ds', how='left')
    forecast = m.predict(future)
    forecast['campaign'] = campaign
    leads.append(forecast[['ds', 'yhat', 'campaign']])
    
leads = pd.concat(leads, ignore_index=True)
leads['yhat'] = leads['yhat'].clip(lower=0)
leads = leads.merge(valid_leads, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(leads['y'], leads['yhat']))
print("WBPE : " , wbpe(leads['y'], leads['yhat']))
print("WCPE : " , wcpe(wape(leads['y'], leads['yhat']), wbpe(leads['y'], leads['yhat']),2))

15:25:10 - cmdstanpy - INFO - Chain [1] start processing
15:25:10 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time


15:25:10 - cmdstanpy - INFO - Chain [1] start processing
15:25:10 - cmdstanpy - INFO - Chain [1] done processing
15:25:10 - cmdstanpy - INFO - Chain [1] start processing
15:25:10 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time


15:25:10 - cmdstanpy - INFO - Chain [1] start processing
15:25:10 - cmdstanpy - INFO - Chain [1] done processing


...

campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek


15:25:11 - cmdstanpy - INFO - Chain [1] start processing
15:25:12 - cmdstanpy - INFO - Chain [1] done processing
15:25:12 - cmdstanpy - INFO - Chain [1] start processing
15:25:12 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng


15:25:13 - cmdstanpy - INFO - Chain [1] start processing
15:25:13 - cmdstanpy - INFO - Chain [1] done processing


WAPE :  0.9047264631632064
WBPE :  0.8451731028880485
WCPE :  0.1376220082961027


In [52]:
prospect = list()
for campaign in train_prospect['campaign'].unique():
    print('campaign:', campaign)
    train_ = train_prospect.loc[train_prospect['campaign'] == campaign]
    valid_ = valid_prospect.loc[valid_prospect['campaign'] == campaign]
    
    m = Prophet(seasonality_mode='additive', yearly_seasonality=False, 
                weekly_seasonality=True, daily_seasonality=True
               )
    m.fit(train_)
    
    future = m.make_future_dataframe(periods=60, include_history=False)
    future = future.merge(valid_[['ds']], on='ds', how='left')
    forecast = m.predict(future)
    forecast['campaign'] = campaign
    prospect.append(forecast[['ds', 'yhat', 'campaign']])
    
prospect = pd.concat(prospect, ignore_index=True)
prospect['yhat'] = prospect['yhat'].clip(lower=0)
prospect = prospect.merge(valid_prospect, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(prospect['y'], prospect['yhat']))
print("WBPE : " , wbpe(prospect['y'], prospect['yhat']))
print("WCPE : " , wcpe(wape(prospect['y'], prospect['yhat']), wbpe(prospect['y'], prospect['yhat']),2))

15:25:13 - cmdstanpy - INFO - Chain [1] start processing
15:25:13 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time


15:25:13 - cmdstanpy - INFO - Chain [1] start processing
15:25:13 - cmdstanpy - INFO - Chain [1] done processing
15:25:13 - cmdstanpy - INFO - Chain [1] start processing
15:25:13 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time


15:25:13 - cmdstanpy - INFO - Chain [1] start processing
15:25:13 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Motor | Discovery | Web 1 | Nationwide
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience


15:25:14 - cmdstanpy - INFO - Chain [1] start processing
15:25:14 - cmdstanpy - INFO - Chain [1] done processing


campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
WAPE :  3.1531389044856035
WBPE :  -2.5713207226674215
WCPE :  nan


In [53]:
funding = list()
for campaign in train_funding['campaign'].unique():
    print('campaign:', campaign)
    train_ = train_funding.loc[train_funding['campaign'] == campaign]
    valid_ = valid_funding.loc[valid_funding['campaign'] == campaign]
    train_['y'] = train_.iloc[-2:, train_.columns.get_loc('y')].fillna(0)

    m = Prophet(seasonality_mode='additive', yearly_seasonality=False, 
                weekly_seasonality=True, daily_seasonality=True
               )
    m.fit(train_)
    
    future = m.make_future_dataframe(periods=60, include_history=False)
    future = future.merge(valid_[['ds']], on='ds', how='left')
    forecast = m.predict(future)
    forecast['campaign'] = campaign
    funding.append(forecast[['ds', 'yhat', 'campaign']])
    
funding = pd.concat(funding, ignore_index=True)
funding['yhat'] = funding['yhat'].clip(lower=0)
funding = funding.merge(valid_funding, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(funding['y'], funding['yhat']))
print("WBPE : " , wbpe(funding['y'], funding['yhat']))
print("WCPE : " , wcpe(wape(funding['y'], funding['yhat']), wbpe(funding['y'], funding['yhat']),2))

campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
WAPE :  0.0
WBPE :  0.0
WCPE :  1.0


In [54]:
NTF = list()
for campaign in train_NTF['campaign'].unique():
    print('campaign:', campaign)
    train_ = train_NTF.loc[train_NTF['campaign'] == campaign]
    valid_ = valid_NTF.loc[valid_NTF['campaign'] == campaign]
    train_['y'] = train_.iloc[-2:, train_.columns.get_loc('y')].fillna(0)
    
    m = Prophet(seasonality_mode='additive', yearly_seasonality=False, 
                weekly_seasonality=True, daily_seasonality=True
               )
    m.fit(train_)
    
    future = m.make_future_dataframe(periods=60, include_history=False)
    future = future.merge(valid_[['ds']], on='ds', how='left')
    forecast = m.predict(future)
    forecast['campaign'] = campaign
    NTF.append(forecast[['ds', 'yhat', 'campaign']])
    
NTF = pd.concat(NTF, ignore_index=True)
NTF['yhat'] = NTF['yhat'].clip(lower=0)
NTF = NTF.merge(valid_NTF, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(NTF['y'], NTF['yhat']))
print("WBPE : " , wbpe(NTF['y'], NTF['yhat']))
print("WCPE : " , wcpe(wape(NTF['y'], NTF['yhat']), wbpe(NTF['y'], NTF['yhat']),2))


campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
WAPE :  0.0
WBPE :  0.0
WCPE :  1.0


In [55]:
result_click = pd.concat([hist_click, click], ignore_index=True)
result_cost = pd.concat([hist_cost, cost], ignore_index=True)
result_leads = pd.concat([hist_leads, leads], ignore_index=True)
result_prospect = pd.concat([hist_prospect, prospect], ignore_index=True)
result_funding = pd.concat([hist_funding, funding], ignore_index=True)
result_NTF = pd.concat([hist_NTF, NTF], ignore_index=True)

In [60]:
Click_SMA = []
ma_window=14

y_hat_sma = result_click[['ds','campaign']]
train_click['ds']= pd.to_datetime(train_click['ds'])
y_hat_sma = pd.merge(y_hat_sma,
                 train_click,
                 left_on=['ds','campaign'], right_on = ['ds','campaign'], 
                 how='left')

for campaign in y_hat_sma['campaign'].unique():
    print('campaign:', campaign)
    train_ = y_hat_sma.loc[y_hat_sma['campaign'] == campaign].reset_index(drop=True)
    
    train_len = len(hist_click.loc[hist_click['campaign'] == campaign])
    valid_ = valid_click.loc[valid_click['campaign'] == campaign]
    #print(train_len)
    
    for x in range(59): #
        train_['sma_forecast'] = train_['y'].rolling(ma_window).mean()
        train_["lag1_sma_forecast"] = (train_.groupby(["campaign"])["sma_forecast"].shift())
        train_.y[train_len:train_len+x+1] = train_.lag1_sma_forecast[train_len:train_len+x+1]
    #print(train_.tail(55))
    
    Click_SMA.append(train_[['ds', 'lag1_sma_forecast', 'campaign']])
    
Click_SMA_df = pd.concat(Click_SMA, ignore_index=True)
Click_SMA_df = Click_SMA_df.merge(valid_click, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(Click_SMA_df['y'], Click_SMA_df['lag1_sma_forecast']))
print("WBPE : " , wbpe(Click_SMA_df['y'], Click_SMA_df['lag1_sma_forecast']))
print("WCPE : " , wcpe(wape(Click_SMA_df['y'], Click_SMA_df['lag1_sma_forecast']), wbpe(Click_SMA_df['y'], Click_SMA_df['lag1_sma_forecast']),2))

result_click = result_click.merge(Click_SMA_df[['ds','campaign','lag1_sma_forecast']], on=['ds', 'campaign'], how='left')


campaign: Leads | BPKB Mobil | Search | Web 1 | Jabodetabek | New
campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
campaign: Leads | BPKB Motor | Discovery | Web 1 | Nationwide
WAPE :  0.1877652955864517
WBPE :  0.18357295120025868
WCPE :  0.8155851212913183


In [61]:
Cost_SMA = []
ma_window=14

y_hat_sma = result_cost[['ds','campaign']]
train_cost['ds']= pd.to_datetime(train_cost['ds'])
y_hat_sma = pd.merge(y_hat_sma,
                 train_cost,
                 left_on=['ds','campaign'], right_on = ['ds','campaign'], 
                 how='left')

for campaign in y_hat_sma['campaign'].unique():
    print('campaign:', campaign)
    train_ = y_hat_sma.loc[y_hat_sma['campaign'] == campaign].reset_index(drop=True)
    
    train_len = len(hist_cost.loc[hist_cost['campaign'] == campaign])
    valid_ = valid_cost.loc[valid_cost['campaign'] == campaign]
    #print(train_len)
    
    for x in range(59): #
        train_['sma_forecast'] = train_['y'].rolling(ma_window).mean()
        train_["lag1_sma_forecast"] = (train_.groupby(["campaign"])["sma_forecast"].shift())
        train_.y[train_len:train_len+x+1] = train_.lag1_sma_forecast[train_len:train_len+x+1]
    #print(train_.tail(55))
    
    Cost_SMA.append(train_[['ds', 'lag1_sma_forecast', 'campaign']])
    
Cost_SMA_df = pd.concat(Cost_SMA, ignore_index=True)
Cost_SMA_df = Cost_SMA_df.merge(valid_cost, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(Cost_SMA_df['y'], Cost_SMA_df['lag1_sma_forecast']))
print("WBPE : " , wbpe(Cost_SMA_df['y'], Cost_SMA_df['lag1_sma_forecast']))
print("WCPE : " , wcpe(wape(Cost_SMA_df['y'], Cost_SMA_df['lag1_sma_forecast']), wbpe(Cost_SMA_df['y'], Cost_SMA_df['lag1_sma_forecast']),2))

result_cost = result_cost.merge(Cost_SMA_df[['ds','campaign','lag1_sma_forecast']], on=['ds', 'campaign'], how='left')

campaign: Leads | BPKB Mobil | Search | Web 1 | Jabodetabek | New
campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
campaign: Leads | BPKB Motor | Discovery | Web 1 | Nationwide
WAPE :  0.29401658276970843
WBPE :  0.28770177757947973
WCPE :  0.7110262400816513


In [62]:
Leads_SMA = []
ma_window=14

y_hat_sma = result_leads[['ds','campaign']]
train_leads['ds']= pd.to_datetime(train_leads['ds'])
y_hat_sma = pd.merge(y_hat_sma,
                 train_leads,
                 left_on=['ds','campaign'], right_on = ['ds','campaign'], 
                 how='left')

for campaign in y_hat_sma['campaign'].unique():
    print('campaign:', campaign)
    train_ = y_hat_sma.loc[y_hat_sma['campaign'] == campaign].reset_index(drop=True)
    
    train_len = len(hist_leads.loc[hist_leads['campaign'] == campaign])
    valid_ = valid_leads.loc[valid_leads['campaign'] == campaign]
    #print(train_len)
    
    for x in range(59): #
        train_['sma_forecast'] = train_['y'].rolling(ma_window).mean()
        train_["lag1_sma_forecast"] = (train_.groupby(["campaign"])["sma_forecast"].shift())
        train_.y[train_len:train_len+x+1] = train_.lag1_sma_forecast[train_len:train_len+x+1]
    #print(train_.tail(55))
    
    Leads_SMA.append(train_[['ds', 'lag1_sma_forecast', 'campaign']])
    
Leads_SMA_df = pd.concat(Leads_SMA, ignore_index=True)
Leads_SMA_df = Leads_SMA_df.merge(valid_leads, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(Leads_SMA_df['y'], Leads_SMA_df['lag1_sma_forecast']))
print("WBPE : " , wbpe(Leads_SMA_df['y'], Leads_SMA_df['lag1_sma_forecast']))
print("WCPE : " , wcpe(wape(Leads_SMA_df['y'], Leads_SMA_df['lag1_sma_forecast']), wbpe(Leads_SMA_df['y'], Leads_SMA_df['lag1_sma_forecast']),2))

result_leads = result_leads.merge(Leads_SMA_df[['ds','campaign','lag1_sma_forecast']], on=['ds', 'campaign'], how='left')

campaign: Leads | BPKB Mobil | Search | Web 1 | Jabodetabek | New
campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
campaign: Leads | BPKB Motor | Discovery | Web 1 | Nationwide
WAPE :  0.0
WBPE :  0.0
WCPE :  1.0


In [63]:
Prospect_SMA = []
ma_window=14

y_hat_sma = result_prospect[['ds','campaign']]
train_prospect['ds']= pd.to_datetime(train_prospect['ds'])
y_hat_sma = pd.merge(y_hat_sma,
                 train_prospect,
                 left_on=['ds','campaign'], right_on = ['ds','campaign'], 
                 how='left')

for campaign in y_hat_sma['campaign'].unique():
    print('campaign:', campaign)
    train_ = y_hat_sma.loc[y_hat_sma['campaign'] == campaign].reset_index(drop=True)
    
    train_len = len(hist_prospect.loc[hist_prospect['campaign'] == campaign])
    valid_ = valid_prospect.loc[valid_prospect['campaign'] == campaign]
    #print(train_len)
    
    for x in range(59): #
        train_['sma_forecast'] = train_['y'].rolling(ma_window).mean()
        train_["lag1_sma_forecast"] = (train_.groupby(["campaign"])["sma_forecast"].shift())
        train_.y[train_len:train_len+x+1] = train_.lag1_sma_forecast[train_len:train_len+x+1]
    #print(train_.tail(55))
    
    Prospect_SMA.append(train_[['ds', 'lag1_sma_forecast', 'campaign']])
    
Prospect_SMA_df = pd.concat(Prospect_SMA, ignore_index=True)
Prospect_SMA_df = Prospect_SMA_df.merge(valid_prospect, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(Prospect_SMA_df['y'], Prospect_SMA_df['lag1_sma_forecast']))
print("WBPE : " , wbpe(Prospect_SMA_df['y'], Prospect_SMA_df['lag1_sma_forecast']))
print("WCPE : " , wcpe(wape(Prospect_SMA_df['y'], Prospect_SMA_df['lag1_sma_forecast']), wbpe(Prospect_SMA_df['y'], Prospect_SMA_df['lag1_sma_forecast']),2))

result_prospect = result_prospect.merge(Prospect_SMA_df[['ds','campaign','lag1_sma_forecast']], on=['ds', 'campaign'], how='left')

campaign: Leads | BPKB Mobil | Search | Web 1 | Jabodetabek | New
campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
campaign: Leads | BPKB Motor | Discovery | Web 1 | Nationwide
WAPE :  0.0
WBPE :  0.0
WCPE :  1.0


In [64]:
Funding_SMA = []
ma_window=14

y_hat_sma = result_funding[['ds','campaign']]
train_funding['ds']= pd.to_datetime(train_funding['ds'])
y_hat_sma = pd.merge(y_hat_sma,
                 train_funding,
                 left_on=['ds','campaign'], right_on = ['ds','campaign'], 
                 how='left')

for campaign in y_hat_sma['campaign'].unique():
    print('campaign:', campaign)
    train_ = y_hat_sma.loc[y_hat_sma['campaign'] == campaign].reset_index(drop=True)
    
    train_len = len(hist_funding.loc[hist_funding['campaign'] == campaign])
    valid_ = valid_funding.loc[valid_funding['campaign'] == campaign]
    #print(train_len)
    
    for x in range(59): #
        train_['sma_forecast'] = train_['y'].rolling(ma_window).mean()
        train_["lag1_sma_forecast"] = (train_.groupby(["campaign"])["sma_forecast"].shift())
        train_.y[train_len:train_len+x+1] = train_.lag1_sma_forecast[train_len:train_len+x+1]
    #print(train_.tail(55))
    
    Funding_SMA.append(train_[['ds', 'lag1_sma_forecast', 'campaign']])
    
Funding_SMA_df = pd.concat(Funding_SMA, ignore_index=True)
Funding_SMA_df = Funding_SMA_df.merge(valid_funding, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(Funding_SMA_df['y'], Funding_SMA_df['lag1_sma_forecast']))
print("WBPE : " , wbpe(Funding_SMA_df['y'], Funding_SMA_df['lag1_sma_forecast']))
print("WCPE : " , wcpe(wape(Funding_SMA_df['y'], Funding_SMA_df['lag1_sma_forecast']), wbpe(Funding_SMA_df['y'], Funding_SMA_df['lag1_sma_forecast']),2))

result_funding = result_funding.merge(Funding_SMA_df[['ds','campaign','lag1_sma_forecast']], on=['ds', 'campaign'], how='left')

campaign: Leads | BPKB Mobil | Search | Web 1 | Jabodetabek | New
campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
WAPE :  0.0
WBPE :  0.0
WCPE :  1.0


In [65]:
NTF_SMA = []
ma_window=14

y_hat_sma = result_NTF[['ds','campaign']]
train_NTF['ds']= pd.to_datetime(train_NTF['ds'])
y_hat_sma = pd.merge(y_hat_sma,
                 train_NTF,
                 left_on=['ds','campaign'], right_on = ['ds','campaign'], 
                 how='left')

for campaign in y_hat_sma['campaign'].unique():
    print('campaign:', campaign)
    train_ = y_hat_sma.loc[y_hat_sma['campaign'] == campaign].reset_index(drop=True)
    
    train_len = len(hist_NTF.loc[hist_NTF['campaign'] == campaign])
    valid_ = valid_NTF.loc[valid_NTF['campaign'] == campaign]
    #print(train_len)
    
    for x in range(59): #
        train_['sma_forecast'] = train_['y'].rolling(ma_window).mean()
        train_["lag1_sma_forecast"] = (train_.groupby(["campaign"])["sma_forecast"].shift())
        train_.y[train_len:train_len+x+1] = train_.lag1_sma_forecast[train_len:train_len+x+1]
    #print(train_.tail(55))
    
    NTF_SMA.append(train_[['ds', 'lag1_sma_forecast', 'campaign']])
    
NTF_SMA_df = pd.concat(NTF_SMA, ignore_index=True)
NTF_SMA_df = NTF_SMA_df.merge(valid_NTF, on=['ds', 'campaign'], how='left')
print("WAPE : " , wape(NTF_SMA_df['y'], NTF_SMA_df['lag1_sma_forecast']))
print("WBPE : " , wbpe(NTF_SMA_df['y'], NTF_SMA_df['lag1_sma_forecast']))
print("WCPE : " , wcpe(wape(NTF_SMA_df['y'], NTF_SMA_df['lag1_sma_forecast']), wbpe(NTF_SMA_df['y'], NTF_SMA_df['lag1_sma_forecast']),2))

result_NTF = result_NTF.merge(NTF_SMA_df[['ds','campaign','lag1_sma_forecast']], on=['ds', 'campaign'], how='left')

campaign: Leads | BPKB Mobil | Search | Web 1 | Jabodetabek | New
campaign: Leads | BPKB Mobil | Search | Web 1 | Nationwide | Targeting | General
campaign: Leads | BPKB Motor | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Jabodetabek
campaign: Leads | Rumah | Search | Web 1 | Jabodetabek
campaign: Leads | BPKB Mobil | Search | Web 1 | Jabarteng
campaign: Leads | BPKB Motor | Search | Web 1 | Nationwide | All Time | New Audience
campaign: Leads | Seritifikat Rumah | Discovery | Web 1 | Cover Area | All Time
campaign: Leads | BPKB Mobil | Discovery | Web 1 | Nationwide | All Time
WAPE :  0.0
WBPE :  0.0
WCPE :  1.0


In [66]:
result_click['metrics'] = 'Click'
result_cost['metrics'] = 'Cost'
result_leads['metrics'] = 'Leads'
result_prospect['metrics'] = 'Prospect'
result_funding['metrics'] = 'Funding'
result_NTF['metrics'] = 'NTF'
forecast_list = [result_click, result_cost, result_leads, result_prospect, result_funding, result_NTF]
forecast_google = pd.concat(forecast_list)
forecast_google['source'] = 'Google'
forecast_google['ad_name'] = ''
forecast_google['adset_name'] = ''
forecast_google = forecast_google.merge(active[['campaign','product']], on=['campaign'], how='left')
forecast_google.rename(columns = {'ds':'date', 'y':'actual','yhat':'forecast','lag1_sma_forecast':'baseline'}, inplace = True)
forecast_google = forecast_google.reindex(['source','product','metrics','campaign','ad_name','adset_name','date','actual','forecast','baseline'], axis=1)
forecast_google.head()

,source,product,metrics,campaign,ad_name,adset_name,date,actual,forecast,baseline
0,Google,BPKB Mobil,Click,Leads | BPKB Mobil | Search | Web 1 | Jabodeta...,,,2023-06-05,36.0,NaN,NaN
1,Google,BPKB Mobil,Click,Leads | BPKB Mobil | Search | Web 1 | Nationwi...,,,2023-06-05,NaN,NaN,NaN
2,Google,BPKB Motor,Click,Leads | BPKB Motor | Search | Web 1 | Jabarteng,,,2023-06-05,154.0,NaN,NaN
3,Google,BPKB Motor,Click,Leads | BPKB Motor | Search | Web 1 | Jabodetabek,,,2023-06-05,88.0,NaN,NaN
4,Google,Sertifikat Rumah,Click,Leads | Rumah | Search | Web 1 | Jabodetabek,,,2023-06-05,189.0,NaN,NaN


In [23]:
#forecast_google.to_csv('forecast_google.csv', index=False)